In [1]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [2]:
using LinearAlgebra, Optim
using BenchmarkTools, Distributions, CSV, DataFrames

In [3]:
ω_c = 5
ω_q = 2
χ = 3e-3
N = 6
amp_c = 1
amp_q = 1

1

In [4]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
X_gate = kron(x_gate(N, Array[[3,4]]), I_qubit)

#X_gate = kron(x_gate(N, Array[[2,3], [4,5]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[3,4]], θ/2), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/4), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])

ψ_initial = rand(N)
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

12-element Vector{Float64}:
 0.0
 0.15294451104880205
 0.0
 0.299992814512568
 0.0
 0.33202705836570584
 0.0
 0.060334877211235795
 0.0
 0.6558518969579372
 0.0
 0.5853104408281705

In [5]:
ψ_final_u = Udue * ψ_initial
ψ_final_x = RX * ψ_initial
ψ_final_x_gate = X_gate * ψ_initial

12-element Vector{Float64}:
 0.0
 0.15294451104880205
 0.0
 0.299992814512568
 0.0
 0.060334877211235795
 0.0
 0.33202705836570584
 0.0
 0.6558518969579372
 0.0
 0.5853104408281705

In [6]:
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 1 0], [1 0 … 0 0; 0 -1 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 -1])

In [7]:
disp(α) = exp((α * adag - α' * a))
ECD(β) = disp(β / 2) * sp + disp(- β / 2) * sm
ROT(ϕ, θ) = exp(-1im * θ / 2 * ( sx * cos(ϕ) + sy * sin(ϕ) )) 
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)

snap (generic function with 1 method)

In [8]:
function cost(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function cost_ecd_rot_ecd(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ECD(β_i[i] + β_r[i] * 1im) * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)'
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy, unitary, N) - cost_before_increment) / h
    end

    return gradient

end

function gate_composition(coefficients, N, l_coeff)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return propagator
    
end

function cost_state_prep(coefficients, N, ψ_n, ψ_m)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(ψ_m' * propagator * ψ_n)^2
    
end


function gradient_state_prep(coefficients, N, h, cost_before_increment, ψ_n, ψ_m)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost_state_prep(coeffs_copy, N, ψ_n, ψ_m) - cost_before_increment) / h
    end

    return gradient

end

gradient_state_prep (generic function with 1 method)

In [9]:
snapgate = snap(rand(N))

12×12 Matrix{ComplexF64}:
 0.654584-0.755989im       0.0-0.0im       …       0.0+0.0im
      0.0-0.0im       0.654584-0.755989im          0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im       …       0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im       …       0.0-0.0im
      0.0+0.0im            0.0+0.0im          0.987003-0.1607im

In [10]:
f(x) = cost(x, RX, N * 2)
h(x) = cost(x, X_gate, N*2)
g(x) = cost_state_prep(x, N * 2, ψ_initial, ψ_final_x_gate)

g (generic function with 1 method)

In [13]:
costs = []
for depth in 36:6:36
    it_costs = []
    for iteration in 1:1
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # opt_coefficients = [0.13127246520334743, -0.019497584668193676, 0.08441258787959842, 0.18853144018033427, 2.4773268321055526, -0.14068538254906235, 0.1067057733639206, 0.1349017917477391, -0.28172200750776616, 2.3343572457141644, 0.23803460000361065, 0.02224019699673719, -0.02819220072444795, 0.14025366097313285, -0.039304691135796384, 0.034017130408503904, -1.270709512762515, 1.1222318350735496, -0.5050327642586442, 0.44136685654889646, -0.11819305256693396, -0.0171689224126802, 0.09552182521389328, -0.2328982824584922, -0.18887578187905554, -0.15747221392930363, 0.048819896470870795, 0.06658477533856631, -0.22675451226200594, -0.0731322584470268, 1.0320426049171438, 0.6225620315319085, 1.568514654834306, 0.7262178776748648, -0.22834995134017863, 0.3779892846129676, 0.4262227353912679, 0.3196753483381889, 0.2953992663817206, -0.19218722459895415, 1.0202317022438367, 1.705913290128121, 0.6088975228114619, 1.3686579107788293, -0.08953083032928448, 1.3785033310248134, 0.6210712278453568, -0.546707314086662, 0.5554710674453178, -0.738159834335507, 0.8480966018787588, 1.3078085508366757, 0.8089514554005326, -1.469472380398311, 0.18028304122964178, 0.17800584503049802, -0.16671991762565194, 0.15198499049007122, -0.04092766820003216, 0.11711271102437686, 0.5188232840958972, -1.7867718434727076, 3.5223065386348553, 2.2224437866146936, -0.7912271654827151, 0.8138625856913634, 0.8251321109743369, 0.3908880448666587, -1.0196639363402769, -0.47085490582037853, 0.4599953976090268, 2.482555446317384, -1.8990959008772232, -2.347458571630002, 1.8940454195441299, 1.6098827996499168, 0.6858333434815906, 3.099926769099066, 0.26105209891049624, 3.116263068234298, 2.5252126200803366, 0.6579487830800986, -2.0398634955596138, -2.063231611559297, 0.17200838633826052, 0.21746703903761155, -0.08632133913870561, 0.10226589649918397, 0.11017725158881246, -0.14735488542926817, -1.1227719271090217, 2.528691011685489, -3.6943677736137874, 0.4459977262437821, 2.4117078989762133, 0.26322261352850806, 1.2256689177520266, -2.2308581588753373, 0.9892257825254104, 0.8281524197559402, 1.9598978630167818, -2.1148746299260353, -2.803899573953217, -2.245508040936566, -0.6193170598374295, 1.7795990385359928, -1.8558619160741494, 1.1841888451815246, 0.16682143048943704, 0.2751262646174936, -1.8060262029518621, 0.1641950718415823, -0.5043898627135203, 2.2137233254732416, -0.3804855296494221, 0.29075602987498295, -0.013080790874542322, -0.052115930912448594, 0.0071533840307411774, 0.023714329530033218]
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/4)]..., (zeros(Int(l_coeff / 4 - length(opt_coefficients)/4)))..., opt_coefficients[Int(length(opt_coefficients)/4)+1:Int(length(opt_coefficients)/2)]..., (zeros(Int(l_coeff / 4 - length(opt_coefficients)/4)))...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:Int(3*length(opt_coefficients)/4)]..., zeros(Int(l_coeff / 4 - length(opt_coefficients)/4))..., opt_coefficients[Int(3*length(opt_coefficients)/4)+1:end]..., zeros(Int(l_coeff / 4 - length(opt_coefficients)/4))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        coefficients = [0.6222945095839462, 0.02661832659671314, 1.4666605514749638, -0.381190423582094, 0.021640862872069188, -0.42410046571181953, 0.41753365764377515, 0.054928627799180126, -0.017432471976456886, 0.1064600558392405, -0.35148256799727406, -0.12543138207581803, -0.5809787252793497, 0.32772161758112, 0.4081674834563646, 0.09993581211392769, 0.28475183114673003, 0.4469064862200441, -0.7006223560516779, 0.27148581331854876, 0.13379843301413094, 0.15973320717246967, -0.13478633065448312, 0.06388981191365549, -0.6555092308200918, -0.03906624801831098, -0.13861842735566296, -0.03309894691166515, 0.015093211482975788, 0.11818969090120247, -0.16371761261693132, -0.4754838543003341, -0.12481839081501309, 0.2862919302800054, -1.7059303994619286, -0.6290314548852712, 0.8081805730547709, 0.0051171806893997316, -0.12778201072580397, -0.23852284099058138, -0.7726419079221978, -1.1028634999567404, 0.9973712914756572, -0.2945632394838389, 0.4649526938710292, -0.909153101353455, -1.8025603746576297, 5.448405834591972e-5, -0.11047558491061286, -0.25036284770324746, 0.01296649373967262, -0.3906853928291418, -0.11430295634851166, 0.2818713625186232, -1.1199893634566527, -0.09821517804524289, 0.09344566800042022, 1.158029062211003, 1.1489313842118984, 0.4361282418025033, 0.6233413795377302, -0.09474929078829741, 0.4371849550656804, -0.04043210063361065, 0.498968659628848, 0.058278047187151176, -0.2683235329345583, 0.5899891881890179, 0.21540302331406797, 0.7327586487996486, 0.093179242726388, -0.8073928825135732, 3.193048085877277, -2.0089084851133383, -2.4409266339442253, 0.21372772856405384, -1.222211659995898, 1.0950426921861776, -2.9970888511321787, 2.726392107201983, -3.3418511977466734, 2.4065038304521784, 0.5085921172563672, 1.5883657402440685, 1.7904885746860364, 0.7951952700422378, 0.2556525392246512, 3.2317120961259977, -2.2422447976827224, 2.818552635038096, 2.7109867780854477, 3.233125824970709, -1.0459408609833485, -1.3865256445669998, -0.013821105442567528, 0.3686623674240935, -2.3700508116449286, 1.4437622270223684, 2.974838437238838, 0.49293253691859773, -2.492499280164086, -1.4543421838693604, 2.1436648788553625, 0.8397799575973875, 2.575876659389223, -1.2060547757910465, 0.7572966587296305, 1.6833511029177348, -2.988566475006429, -1.1251826676732695, -0.060605722333071536, -2.887809199570042, 1.7847703238088126, -1.6947042616462709, 3.23197894358413, 1.2979689486153814, -3.5843089373101953, -0.21412680246758833, -2.164112610862576, 1.5823487630918651, -1.532823596642582, -0.5245807985488108, -2.0944401992290045, -2.4418949261732203, 1.2773696698469543, -0.6992870577763725, -1.1061795889297046, 3.0086313905606703, -1.7953385598296754, 2.4286088582487633, 0.4017851063321946, 0.35466864455654384, 0.9220178961878963, 1.036475675989083, -3.296741960300431, 1.2573530097605548, 2.619877929300715, -1.8919111277716034, 1.903732587886861, -0.9931940365894778, 2.3291817489167492, -1.6018968898277182, -1.4523153786844014, 1.4652833028181464]
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100, iterations = 500))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 36 iterations :1
Iter     Function value   Gradient norm 


     0     6.516648e-04     7.684787e-04
 * time: 0.00011515617370605469


   100     6.045524e-04     6.159594e-04
 * time: 176.33599495887756


   200     5.568568e-04     3.335549e-04
 * time: 359.4869010448456


   300     5.034936e-04     6.458806e-04
 * time: 539.7624220848083


   400     4.589750e-04     5.018713e-04
 * time: 721.7998390197754


InterruptException: InterruptException:

In [62]:
it_costs_x = []
coefficients_x = []
it_costs_rx = []
coefficients_rx = []
blocks = []
for depth in 36:6:36
    for iteration in 1:10
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # opt_coefficients = [0.0976423458354245, 0.0450708234683308, 0.10976747963502345, 0.08416609504894443, 2.551326070782372, -0.08964024119320309, 0.10485248324424644, 0.13969513913978635, -0.22349455300798343, 2.3735987675524037, 0.19185222571130106, 0.09817439361481875, 0.10464682496970999, 0.03769605958097719, 0.059524186293775296, 0.008378579058631288, -1.2534771254122576, 1.1903829602499862, 0.016016288408910853, -0.05483313987446702, -0.046034209736723807, -0.027533341989763437, 0.006811587319806202, -0.07449680205270068, 0.9909477420650091, 0.7105671478179529, 1.4506719329759887, 0.6584572437091945, -0.1462197067998505, 0.3890746504254293, 0.4550190349625619, 0.27485980171484087, 0.24866482812659083, -0.13476539605421262, 1.1225574211169433, 1.9743342171260378, 1.0562819850952958, 0.7320395937688988, 0.4678336590764824, 0.7663663217427629, 0.7493470855455572, -0.6802045420602124, 0.5414009020842547, -0.5236457353726556, 0.7006096048990366, 1.288999517730632, 0.33766541989560206, -1.3141536095001372, 1.2548422513237936, -1.4739003495829288, 3.612006758803497, 2.177902858706418, -0.52269040375497, 0.9564247538849191, 0.49002185275320925, 0.47233587796819054, -0.7410940755657739, -0.41486097094262636, 0.42027037104216425, 2.178242513687377, -1.526022531001938, -2.559186549835382, 2.114662728368936, 2.087936066989978, 0.7610469841037554, 3.1090098597892273, 0.4006039324836421, 3.1069649976511053, 1.8012672154816713, 0.5695265489909448, -2.77394600557009, -1.5567065154837294, -0.9443614233970088, 2.2647812845224053, -3.57309947272525, 0.5409027672256352, 2.425316199161032, 0.16790448062740732, 1.1584460466449906, -2.4993058076042263, 0.8206079200349031, 0.7528221404227818, 1.353149399622874, -2.1365245400174433, -2.372828756091764, -2.5847407449073145, -0.6423548068234998, 1.8509304528464354, -1.6388536401478402, 1.462231976003448, 0.605305856716617, 1.0233566302172725, -1.7465358497919055, 0.32976779077893226, -0.7074764414799524, 2.9589238120929138]
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/4)]..., (zeros(Int(l_coeff / 4 - length(opt_coefficients)/4)))..., opt_coefficients[Int(length(opt_coefficients)/4)+1:Int(length(opt_coefficients)/2)]..., (zeros(Int(l_coeff / 4 - length(opt_coefficients)/4)))...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:Int(3*length(opt_coefficients)/4)]..., zeros(Int(l_coeff / 4 - length(opt_coefficients)/4))..., opt_coefficients[Int(3*length(opt_coefficients)/4)+1:end]..., zeros(Int(l_coeff / 4 - length(opt_coefficients)/4))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res_x = Optim.optimize(h, coefficients , LBFGS(), Optim.Options())
        res_rx = Optim.optimize(f, coefficients , LBFGS(), Optim.Options())
        push!(blocks, depth)
        push!(it_costs_x,Optim.minimum(res_x))
        push!(it_costs_rx,Optim.minimum(res_rx))
        push!(coefficients_x,Optim.minimizer(res_x))
        push!(coefficients_rx,Optim.minimizer(res_rx))
    end
end

In [ ]:
df_x = DataFrame(fidelity=it_costs_x, blocks=blocks, coefficients=coefficients_x)
df_rx = DataFrame(fidelity=it_costs_rx, blocks=blocks, coefficients=coefficients_rx)
CSV.write("../../data/x_gate.txt", df_x)
CSV.write("../../data/rx_gate.txt", df_rx)

# stoc gradient descent

In [ ]:
costs_final = []
for depth in 64:8:64
    it_costs = []
    for iteration in 1:3
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        coefficients = [0.8690408632149678, -0.7236416401971805, -0.0908719850588177, 0.8258628776740167, 0.9056445769487957, -0.8220939210081029, 0.29601861853616424, 0.7813552229187792, 0.9837594907380149, -0.10686709645389225, -0.6656407250459772, 0.992278727387623, -0.4893461915705191, -1.093993962903741, 0.4845407107960842, -0.33857113975897773, -0.438797994554033, -0.5977365084149797, 0.20827466739249778, -0.8959109454265681, 0.5383716212207733, 1.3219169845803183, 0.8578064725920909, -0.7256810259845291, 0.37256259960242327, -1.2114316020995235, -0.282902962907945, -0.26590734759323503, 0.5548879566071089, 0.6803304120122661, -1.0549066677248888, 0.4013045273205048, 0.039560949181603355, 1.0529793828938236, -0.044290684794351255, -0.21790564460826214, 0.7641852601374161, 0.9310403877890154, -0.24519813978115026, 0.6528372052655407, -1.2521949229101696, -0.21682088705693697, 0.10911919627652335, -0.31451889968452584, 0.07428438971871276, 0.8168118972903352, -0.4306959382326172, -1.31201487771103, -0.5551956081923786, -0.24938592347753555, 0.7261072879386111, 0.3449918944429117, 1.0218022420859423, 0.9586632779953215, 0.9830015979456759, 1.1800344827793208, -0.6015729201693757, 0.703237921035128, -0.8659071933834304, 0.7405471254766851, 0.5502125342538493, -0.1279261558044507, 0.5436405284887846, 0.7561715320031175, -1.8294742583884145, 0.8274849373492085, -0.5023928927013781, -0.2106775708083146, -0.25623844263590434, -0.13700323024471564, -0.5427032401830472, 0.8481733035397572, 2.857123893083541, 0.6722772567617452, 3.3393904892971857, -2.7921664352181588, -1.4927954172559996, 0.3500341652427946, 2.178592973947075, 2.0048971146113073, -1.3350642362244973, 0.9711032282505817, 2.3816703973089663, -2.6138104747765247, 2.5134841942159674, -1.9047337097507626, 2.2260526473073146, -1.8796539188004344, 1.6517968966982153, 1.4620145034237324, 1.8366376463164558, -1.3614199467978896, -1.3067881525922014, -0.8271088962045482, -1.3523594955345797, -0.8216607978872826, -1.015386302411769, -1.4951025575413324, 2.199130944918348, -2.098541424724332, 1.8106459325712736, -2.7528680557326783, 1.434522172097638, 0.23235089810469334, -1.3427653914596476, -2.1865004319117882, -4.499311913607152, 3.5233665940145906, 0.11147763894762594, 1.7253123134995119, 0.3069539608771961, 0.7456729443236856, 0.9675539799184902, -2.0807423878078692, -0.21274877285924562, -2.4625967782327733, -0.8958201449217154, -0.16506248177386307, -0.7781344488152088, 0.5657089824636788, 0.49103015856219095, -1.893089322888448, 1.8586226474868552, 1.3367905090651682, -0.845322993521433, -2.414547922619251, -2.0653040809018646, 2.13700608386267, 0.6560460356595541, 1.2398223292845443, 2.962599533837159, 1.1325322497723143, 3.463842619105753, -3.938053691752728, -0.9569023982656585, 1.605238908260515, 0.6974820952130029, 2.5406564651431, 2.285077612710168, 0.027961409188739638, 1.0736422395589698, 2.4870006619787572, 0.692378916991639, -2.718696870022777]
        beta1 = 0.8
        beta2 = 0.999
        alpha = 0.01
        m = zeros(length(coefficients))
        v = zeros(length(coefficients))
        coeffs_stored = []
        costs = []
        cost_prev_it = cost(coefficients, RX, N*2)  #cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #

        for i in 1:1000
            grad = gradient(coefficients, RX, N * 2, 1e-8, cost_prev_it) #gradient_state_prep(coefficients, N*2, 1e-8, cost_prev_it, ψ_initial, ψ_final_x_gate) #
            for n in 1:length(coefficients)
                m[n] = beta1 * m[n] + ( 1 - beta1 ) * grad[n]
                v[n] = beta2 * v[n] + ( 1 - beta2 ) * grad[n]^2
                m_norm = m[n] / ( 1 - beta1^i )
                v_norm = v[n] / ( 1 - beta2^i )
                coefficients[n] = coefficients[n] - alpha * m_norm / (sqrt(v_norm) + 1e-8)
            end
            cost_prev_it = cost(coefficients, RX, N*2)# cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #
            push!(costs, cost_prev_it)

            if i % 50 == 0
                if norm(grad) < 1e-6
                    break
                end
                println(i)
                println(cost_prev_it)
            end
        end
        println("finished: d = ", depth)
        println(costs[end])
        push!(it_costs,costs[end])
        println(coefficients)
    end
    println(it_costs)
    println(minimum(it_costs))
    push!(costs_final, minimum(it_costs))
end
costs_final